## Calculate ATO for TAZ centroid(s)

1. Run 1_setup_network.ipynb to create NetworkDataset_MM
2. Create OD Cost Matrix Layer
3. Add origin(s)
4. Add destinations
5. Solve
6. Join attributes to solved Lines layer
7. Weight HH and JOB by time decay
8. Sum

In [ ]:
from arcgis.gis import GIS
import arcpy
import os

from arcgis.features import SpatialDataFrame
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor

def survey_weight(t):
    if t <= 3:
        return 1
    elif (t > 3) & (t <= 20):
        return -0.0382 * t + 1.1293
    elif t > 20:
        return 1/(1 + math.exp(0.1092 * t - 1.5604))
    else:
        return 0

# Update this
base_path = r"c:\wfrc\ato"

base_gdb = os.path.join(base_path, "ato.gdb")

arcpy.env.workspace = base_gdb

In [ ]:
# from 1_setup_network.ipynb

# baseline
input_network_dataset = os.path.join(base_gdb, r"NetworkDataset\NetworkDataset_ND")
#input_network_dataset = r"C:\wfrc\ato\shp\MM_NetworkDataset_08052021.gdb\NetworkDataset\NetworkDataset_ND"
    
output_file = "driving_baseline"

# mod
#input_network_dataset = "C:/arcgis/wfrc/mm_mod_5.gdb/NetworkDataset/NetworkDataset_MM"
#output_file = "transit_mod_5.csv"

mode = "Driving" # Driving | Transit

In [ ]:
# Create OD Cost Matrix Layers
arcpy.na.MakeODCostMatrixAnalysisLayer(
    network_data_source = input_network_dataset, 
    layer_name = "OD Cost Matrix", 
    travel_mode = mode, 
    cutoff = 60.0,
    line_shape = "NO_LINES"
)

In [ ]:
# Add Origins
origin_options = "Name CO_TAZID #;TargetDestinationCount # #;CurbApproach # 0;Cutoff_Mins_BikeOnly # #;Cutoff_Mins_BikeTransitOnly # #;Cutoff_Mins_DriveOnly # #;Cutoff_Mins_PedOnly # #;Cutoff_Mins_PedTransitOnly # #;Cutoff_Mins_TransitOnly # #;Cutoff_Miles # #"

arcpy.na.AddLocations(
    in_network_analysis_layer = "OD Cost Matrix",
    sub_layer = "Origins", 
    in_table = os.path.join(base_gdb, "taz_centroids"), 
    field_mappings = origin_options, 
    search_tolerance = "10000 Meters", 
    search_criteria  = [["BikePedAuto","SHAPE"],
                        ["ConnectorNetwork","NONE"],
                       ["NetworkDataset_ND_Junctions","NONE"],
                       ["TransitRoutes","NONE"],
                       ["TransitStops","NONE"]], 
    match_type = "Priority",
    snap_to_position_along_network = "SNAP",
    snap_offset = "0 Meters",
    exclude_restricted_elements = "EXCLUDE"
)

In [ ]:
# Add Destinations
arcpy.na.AddLocations(
    in_network_analysis_layer = "OD Cost Matrix",
    sub_layer = "Destinations", 
    in_table = os.path.join(base_gdb, "taz_centroids"), 
    field_mappings = "Name CO_TAZID #;CurbApproach # 0", 
    search_tolerance = "10000 Meters", 
    search_criteria  = [["BikePedAuto","SHAPE"],
                        ["ConnectorNetwork","NONE"],
                       ["NetworkDataset_ND_Junctions","NONE"],
                       ["TransitRoutes","NONE"],
                       ["TransitStops","NONE"]], 
    match_type = "Priority",
    snap_to_position_along_network = "SNAP",
    snap_offset = "0 Meters",
    exclude_restricted_elements = "EXCLUDE"
)

In [ ]:
%%time
# Solve
arcpy.na.Solve("OD Cost Matrix", "SKIP", "TERMINATE", None, '')

In [ ]:
%%time
od = pd.DataFrame.spatial.from_featureclass(r"OD Cost Matrix\Lines")

In [ ]:
od.tail()

In [ ]:
%%time
od['Origin_CO_TAZID'] = od['Name'].apply(lambda x: int(x.split(' - ')[0]))
od['Dest_CO_TAZID'] = od['Name'].apply(lambda x: int(x.split(' - ')[1]))

In [ ]:
taz = pd.DataFrame.spatial.from_featureclass(os.path.join(base_gdb, "taz_ato"))

In [ ]:
taz = taz[['CO_TAZID', 'HH_19', 'JOB_19', 'JOBAUTO_19', 'HHAUTO_19', 'JOBTRANSIT_19', 'HHTRANSIT_19']]

In [ ]:
df = pd.merge(od, taz, left_on="Dest_CO_TAZID", right_on="CO_TAZID", )

In [ ]:
df.head()

In [ ]:
# Weight outputs

if mode == 'Driving':
    df['travel_time'] = df['Total_Mins_DriveOnly']
elif mode == 'Transit':
    df['travel_time'] = df['Total_Mins_PedTransitOnly']

df['survey_weight'] = df['travel_time'].apply(lambda x: survey_weight(x))
df['survey_weight'] = df['survey_weight'].round(3)

df['weighted_jobs'] = df['survey_weight'] * df['JOB_19']
df['weighted_hh'] = df['survey_weight'] * df['HH_19']
df['weighted_jobs'] = round(df['weighted_jobs'])
df['weighted_hh'] = round(df['weighted_hh'])

df['ato'] = df['weighted_jobs'] + df['weighted_hh']

In [ ]:
# write to disk
df[['Name', 'Origin_CO_TAZID', 'Dest_CO_TAZID', 'travel_time', 'survey_weight',
         'weighted_jobs', 'weighted_hh', 'ato']].to_csv(output_file + '.csv')

In [ ]:
taz_summary = df.groupby('Origin_CO_TAZID').agg(
    jobs=pd.NamedAgg(column='weighted_jobs', aggfunc=sum),
    hh=pd.NamedAgg(column='weighted_hh', aggfunc=sum)
)
taz_summary['ato'] = taz_summary['jobs'] + taz_summary['hh']
taz_summary.to_csv(output_file + '_summary.csv')